# Data Science 100 Knocks (Structured Data Processing) - Python



## 参照(Reference) : 「データサイエンティスト協会スキル定義委員」の「データサイエンス100本ノック（構造化データ加工編）」

## The Data Scientist Society Github : https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess

## Data Science 100 Knocks (Structured Data Processing) URL : https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess/blob/master/docker/work/preprocess_knock_Python.ipynb

- Note: This is an ipynb file originally created by The Data Scientist Society(データサイエンティスト協会スキル定義委員) and translated from Japanese to English by DeepL.

- The reason I updated this file is to spread this practice, which is useful for everyone who wants to practice Python, from beginners to advanced engineers. 

- Since this data was created for Japanese, you may face language problems when practicing. But do not worry, it will not affect much.

## Getting Started
- First, run the following cells
- Import the necessary libraries and read data from the database (PostgreSQL)
- The following cell imports libraries that are expected to be used, such as pandas.
- If there are other libraries you wish to use, install them as needed (you can also install them by "!pip install library name").
- The process can be divided into multiple times.
- Names, addresses, etc. are dummy data and are not real.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler


if 'PG_PORT' in os.environ:
    pgconfig = {
        'host': 'db',
        'port': os.environ['PG_PORT'],
        'database': os.environ['PG_DATABASE'],
        'user': os.environ['PG_USER'],
        'password': os.environ['PG_PASSWORD'],
    }


    # Connector for pd.read_sql
    conn = psycopg2.connect(**pgconfig)

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }

    df_customer = pd.read_csv("data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("data/category.csv", dtype=dtype)
    df_product = pd.read_csv("data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("data/geocode.csv", dtype=dtype)


Cloning into '100knocks-preprocess'...
remote: Enumerating objects: 1571, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 1571 (delta 52), reused 109 (delta 38), pack-reused 1434
Receiving objects: 100% (1571/1571), 20.79 MiB | 7.96 MiB/s, done.
Resolving deltas: 100% (831/831), done.


# Exercise

---
> P-001: Display the first 10 items of all items from the receipt details data (df_receipt) and visually check what data is held.

---
> P-002: Display 10 items from the receipt details data (df_receipt), specifying the columns in the order of sales_date (sales_ymd), customer_id (customer_id), product_code (product_cd) and sales amount (amount).

---
> P-003: From the receipt details data (df_receipt), specify the columns in order of sales_date (sales_ymd), customer_id (customer_id), product_code (product_cd) and sales_amount (amount) and display 10 items. However, extract while renaming the items from sales_ymd to sales_date.

---
> P-004: From receipt details data (df_receipt), specify columns in the order of sales_date (sales_ymd), customer ID (customer_id), product code (product_cd) and sales amount (amount) and extract data satisfying the following conditions.

> - Customer ID (customer_id) is "CS018205000001"

---
> P-005: Extract data from receipt details data (df_receipt) specifying columns in the order of sales_date (sales_ymd), customer ID (customer_id), product code (product_cd) and sales amount (amount) and satisfying all the following conditions.
> - Customer ID (customer_id) is "CS018205000001".
> - Sales amount (amount) is 1,000 or more.

---
> P-006: Extract data from receipt details data (df_receipt) ,specifying the columns in the order of sales_date (sales_ymd), customer ID (customer_id), product code (product_cd), sales quantity (quantity) and sales amount (amount) and satisfying all the following conditions.
> - Customer ID (customer_id) is "CS018205000001".
> - Sales amount (amount) is 1,000 or more or sales quantity (quantity) is 5 or more.

---
> P-007: From the receipt details data (df_receipt), specify the columns in order of sales date (sales_ymd), customer ID (customer_id), product code (product_cd) and sales amount (amount) and extract data that satisfy all the following conditions.
> - Customer ID (customer_id) is "CS018205000001".
> - Sales amount (AMOUNT) is between 1,000 and 2,000.

---
> P-008: From the receipt details data (df_receipt), specify the columns in order of sales date (sales_ymd), customer ID (customer_id), product code (product_cd) and sales amount (amount), and extract data satisfying all the following conditions.
> - Customer ID (customer_id) is "CS018205000001".
> - Product code (product_cd) is other than "P071401019".

---
> P-009: In the following process, rewrite OR to AND without changing the output result.
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

---
> P-010: From the shop data (df_store), extract all items whose shop code (store_cd) starts with "S14" and display 10 items.

---
> P-011: From the customer data (df_customer), extract all items whose customer ID (customer_id) ends with 1, and display 10 items.

---
> P-012: From shop data (df_store), display all items with "横浜市(Yokohama-city)" in the address.

---
> P-013: From the customer data (df_customer), extract all data whose status code (status_cd) begins with the letters A to F and display 10 items.

---
> P-014: From the customer data (df_customer), extract all data items whose status code (status_cd) ends with the numbers 1 to 9, and display 10 items.

---
> P-015: From the customer data (df_customer), extract all data items whose status code (status_cd) begins with the letters A to F and ends with the numbers 1 to 9, and display 10 items.

---
> P-016: From the shop data (df_store), display all items of data where the telephone number (tel_no) is 3 digits - 3 digits - 4 digits.

---
> P-017: Sort customer data (df_customer) by birth_day(birth_day) in order of age and display all 10 items from the top.

---
> P-018: Sort customer data (df_customer) by birth_day(birth_day) in descending order, displaying all 10 items from the top.

---
> P-019: Rank the receipt details data (df_receipt) in descending order of sales amount per item (amount) and display 10 items from the top. The items shall display the customer ID (customer_id), sales amount (amount) and the rank assigned. If the sales amount (amount) is equal, the same rank shall be assigned.

---
> P-020: For receipt details data (df_receipt), assign ranks in descending order of sales amount (amount) per item and display the first 10 items. Items shall display the customer ID (customer_id), sales amount (amount) and the rank assigned. Even if the sales amount (amount) is equal, a different rank should be assigned.

---
> P-021: Count the number of cases against the receipt details data (df_receipt).

---
> P-022: Count the number of unique cases against the customer ID (customer_id) of the receipt details data (df_receipt).

---
> P-023: Sum the sales amount (amount) and sales quantity (quantity) for each shop code (store_cd) in the receipt details data (df_receipt).

---
> P-024: Find the most recent sales date (sales_ymd) for each customer ID (customer_id) for the receipt details data (df_receipt) and display the 10 most recent sales dates (sales_ymd).


---
> P-025: For receipt details data (df_receipt), find the oldest sales date (sales_ymd) for each customer ID (customer_id) and display the 10 results.

---
> P-026: For receipt details data (df_receipt), find the newest sales date (sales_ymd) and the oldest sales date (sales_ymd) for each customer ID (customer_id) and display 10 cases where both are different.

---
> P-027: For the receipt details data (df_receipt), calculate the average of the sales amount (amount) for each shop code (store_cd) and display the top 5 in descending order.

---
> P-028: Calculate the median sales amount (amount) for each shop code (store_cd) for the receipt details data (df_receipt) and display the TOP 5 in descending order.

---
> P-029: For receipt details data (df_receipt), calculate the mode value of the product code (product_cd) for each shop code (store_cd) and display 10 cases.

---
> P-030: Calculate the variance of the sales amount (amount) for each shop code (store_cd) against the receipt details data (df_receipt) and display five cases in descending order.

---
> P-031: Calculate the standard deviation of the sales amount (amount) for each shop code (store_cd) for the receipt details data (df_receipt) and display 5 cases in descending order.

TIPS:

Note that the default values for ddof are different in Pandas and Numpy
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

---
> P-032: Find the percentile values for the sales amount (amount) in the receipt details data (df_receipt) in 25% increments.

---
> P-033: Calculate the average of the sales amount (amount) for each shop code (store_cd) for the receipt details data (df_receipt) and extract those that are 330 or more.

---
> P-034: For the receipt details data (df_receipt), sum the amount of sales (amount) for each customer ID (customer_id) and find the average for all customers. However, exclude customer IDs starting with "Z" as they represent non-members.

---
> P-035: For the receipt details data (df_receipt), sum the sales amount (amout) for each customer ID (customer_id) to obtain the average of all customers, extract the customers who spend more than the average and display 10 items. However, exclude customer IDs starting with "Z" as they represent non-members.

---
> P-036: Combine receipt details data (df_receipt) and shop data (df_store) internally and display all items of receipt details data and store name (store_name) of shop data for 10 items.


---
> P-037: Join product data (df_product) and category data (df_category) internally and display all items of the product data and 10 category sub-category names (category_small_name) of the category data.

---
> P-038: Calculate the total sales amount for each customer from the customer data (df_customer) and receipt details data (df_receipt) and display 10 items. However, for customers with no sales records, the sales amount should be displayed as 0. Customers whose gender code (gender_cd) is female (1) should be included, and non-members (whose customer ID starts with "Z") should be excluded.

---
> P-039: From the receipt details data (df_receipt), create a data set containing the top 20 customers with the highest number of days of sales and a data set containing the top 20 customers with the highest total sales value, respectively, and furthermore, merge these two completely externally. However, exclude non-members (whose customer ID starts with "Z").

---
> P-040: You want to create data combining all shops and all products. Direct product the shop data (df_store) and the product data (df_product) and calculate the number of cases.

---
> P-041: Sum up the sales amount (amount) of the receipt details data (df_receipt) by date (sales_ymd) and calculate the increase or decrease in sales amount from the date of the last sale. And display 10 results.

---
> P-042: Tabulate the sales amount (AMOUNT) of the receipt details data (df_receipt) by date (sales_ymd), and for each date, combine the data of the last, previous and three previous sales dates. Then display 10 results.

---
> P-043: Combine receipt details data (df_receipt) and customer data (df_customer) to create sales summary data, summing the sales amount (amount) by gender code (gender_cd) and age (calculated from age). Gender_cd should be 0 for male, 1 for female and 9 for unknown.

>
> However, the item structure should consist of four items: age, female sales amount, male sales amount and sales amount of unknown gender (vertical cross tabulation of age and horizontal cross tabulation of gender). In addition, the age shall be in deciles of 10 years.

---
> P-044: The sales summary data (df_sales_summary) created in 043 had sales by gender carried horizontally. From this data, convert the gender into three items: age, gender code and sales amount, by having the gender carried vertically. The gender code should be "00" for male, "01" for female and "99" for unknown.

---
> P-045: The date of birth (birth_day) in the customer data (df_customer) is held in date type. Convert this to a string in YYYYMMDD format and display 10 cases together with the customer ID (customer_id).

---
> P-046: The application date (application_date) of the customer data (df_customer) holds data in a string type in YYYYMMDD format. Convert this to a date type and display 10 cases together with the customer ID (customer_id).

---
> P-047: The sales date (sales_ymd) of the receipt details data (df_receipt) holds data in numeric type in YYYYMMDD format. Convert this to a date type and display 10 items together with the receipt number (receipt_no) and receipt sub-number (receipt_sub_no).


---
> P-048: The sales epoch seconds (sales_epoch) of the receipt details data (df_receipt) holds data in UNIX seconds of numeric type. Convert it to date type and display 10 cases together with receipt number (receipt_no) and receipt sub-number (receipt_sub_no).

---
> P-049: Convert the sales epoch seconds (sales_epoch) of the receipt details data (df_receipt) to date type, extract only the "年(year)" and display 10 items with receipt number (receipt_no) and receipt sub-number (receipt_sub_no).

---
> P-050: Convert the sales epoch seconds (sales_epoch) of the receipt details data (df_receipt) to date type, extract only the "月(month)" and display 10 items with receipt number (receipt_no) and receipt sub number (receipt_sub_no). Note that the "月(month)" should be retrieved with two digits filled with zero.

---
> P-051: Convert the sales epoch seconds of the receipt details data (df_receipt) to date type, extract only the "日(day)" and display 10 items with receipt number (receipt_no) and receipt sub-number (receipt_sub_no). Note that the "日(day)" should be extracted with two digits filled with zero.

---
> P-052: Sum the sales amount (amount) of the receipt details data (df_receipt) for each customer ID (customer_id), binaryise the total sales amount to 0 for amounts less than 2,000 yen and 1 for amounts greater than 2,000 yen, and display 10 items with the customer ID and total sales amount. Display 10 items together with the customer ID and the total sales amount. However, exclude customer IDs starting with "Z" as they represent non-members.

---
> P-053: Binarise the postcode (postal_cd) of the customer data (df_customer) to 1 for Tokyo (the first three digits are 100 to 209) and 0 for all others. Further, combine it with the receipt details data (df_receipt) and count the number of customers with sales results for the entire period, for each binary value created.

---
> P-054: The address of the customer data (df_customer) is one of 埼玉県(Saitama-prefecture), 千葉県(Chiba-prefecture), 東京都(Tokyo-Metropolitan) or 神奈川県(Kanagawa-prefecture). Create a code value for each prefecture, and display 10 cases together with the customer ID and address. The values should be 11 for 埼玉県(Saitama-prefecture), 12 for 千葉県(Chiba-prefecture), 13 for 東京都(Tokyo-Metropolitan) and 14 for 神奈川県(Kanagawa-prefecture).

---
> P-055: Sum the sales amount (amount) of the receipt details (df_receipt) data for each customer ID (customer_id) and find the quartile point of the total amount. Then, create category values for the total sales amount for each customer based on the following criteria, and display the 10 results together with the customer ID and the total sales amount. The category values shall be 1-4 in order.

>
> - Minimum value and above and below the first quartile - Assign 1.
> - First quartile and above but below the second quartile - Assign 2.
> - Second quartile and above but below the third quartile - Assign 3.
> - Third quartile and above - Assign 4.

---
> P-056: Calculate the age in 10-year age increments based on the age (age) of the customer data (df_customer) and display 10 cases together with the customer ID (customer_id) and date of birth (birth_day). However, all age groups above 60 years should be assumed to be in the 60s. The name of the category representing the age is optional.

---
> P-057: Using the extraction results of 056 and the gender code (gender_cd), create new category data representing the combination of gender x age and display 10 cases. The value of the category representing the combination shall be arbitrary.

---
> P-058: Create a dummy variable for the gender code (gender_cd) of the customer data (df_customer) and display 10 cases together with the customer ID (customer_id).


---
> P-059: Sum the sales amount (AMOUNT) of the receipt details data (df_receipt) for each customer ID (customer_id), standardise the total sales amount to mean 0 and standard deviation 1 and display 10 cases with the customer ID and total sales amount. The standard deviation used for standardisation may be either the square root of the variance or the square root of the unbiased variance. However, exclude customer IDs starting with "Z", as they represent non-members.

TIPS:
- The query() argument engine allows you to choose between 'python' and 'numberxpr', defaulting to numberxpr if installed, otherwise python is used. Furthermore, string methods cannot be used in query() unless engine='python'.


---
> P-060: Sum the sales amount (AMOUNT) of the receipt details data (df_receipt) for each customer ID (customer_id), normalise the total sales amount to a minimum value of 0 and a maximum value of 1 and display 10 items with the customer ID and total sales amount. However, exclude customer IDs starting with "Z" as they represent non-members.


---
> P-061: Sum the sales amount (amount) of the receipt details data (df_receipt) for each customer ID (customer_id), convert the total sales amount to ordinary logarithm (bottom 10) and display 10 items with customer ID and total sales amount. However, exclude customer IDs starting with "Z" as they represent non-members.

---
> P-062: Sum the sales amount (amount) of the receipt details data (df_receipt) for each customer ID (customer_id), convert the total sales amount to the natural logarithm (bottom e) and display 10 items with the customer ID and total sales amount. However, exclude customer IDs starting with "Z" as they represent non-members.

---
> P-063: Calculate the profit amount of each product from the unit price(unit price) and cost of the product(unit_cost) data (df_product) and display 10 results.

---
> P-064: Calculate the overall average profit margin for each product from the unit price (unit_price) and cost (unit_cost) of the product data (df_product). Note, however, that unit_price and cost are missing.

---
> P-065: For each product in the product data (df_product), find the new unit price that gives a profit margin of 30%. However, round down to the nearest yen. Then display 10 results and confirm that the profit margin is approximately around 30%. Note that there is a deficit in unit_price（unit_price） and unit_cost（unit_cost）.

---
> P-066: For each product in the product data (df_product), find a new unit price that gives a profit margin of 30%. This time, round to the nearest yen (rounding or rounding to even numbers is fine). Then display 10 results and check that the profit margin is around 30%. Note, however, that there are deficiencies in unit_price（unit_price） and unit_cost（unit_cost）.


---
> P-067: For each product in the product data (df_product), find a new unit price that gives a profit margin of 30%. This time, round up to the nearest yen. Then display 10 results and check that the profit margin is around 30%. Note, however, that there is a deficit in the unit price (unit_price) and cost (unit_cost).


---
> P-068: For each product in the product data (df_product), find the amount including tax at a consumption tax rate of 10%, rounding down fractions of a yen and displaying 10 results. Note that the unit price (unit_price) is missing.

---
> P-069: Combine receipt details data (df_receipt) and product data (df_product), calculate the total sales value of all products for each customer and the total sales value of products whose category major classification code (category_major_cd) is "07" (bottled canned food), and find the ratio between the two. Only customers with a sales record for category major category code "07" (bottled canned food) should be selected, and 10 results should be displayed.


---
> P-070: Calculate the number of days elapsed from the membership application date (application_date) of the customer data (df_customer) against the sales date (sales_ymd) of the receipt details data (df_receipt) and display 10 items with the customer ID (customer_id), sales date and membership application date. (Note that sales_ymd is numeric and application_date is a string).

---
> P-071: Calculate the number of months elapsed from the membership application date (application_date) of the customer data (df_customer) against the sales date (sales_ymd) of the receipt details data (df_receipt), and display 10 items with customer ID (customer_id), sales date and membership application date (Note that sales_ymd is a number and application_date is a string).

---
> P-072: Calculate the number of years elapsed from the membership application date (application_date) of the customer data (df_customer) against the sales date (df_customer) of the receipt details data (df_receipt), and display 10 items with customer ID (customer_id), sales date and membership application date (Note that sales_ymd is a number and application_date is a string).


---
> P-073: Calculate the elapsed time in epoch seconds from the membership application date (application_date) of the customer data (df_customer) against the sales date (sales_ymd) of the receipt details data (df_receipt), and display 10 items with customer ID (customer_id), sales date and (Note that sales_ymd is held as a number and application_date as a string). Note that no time information is held, so each date should represent 0:00:00:00.

---
> P-074: For the sales date (sales_ymd) of the receipt details data (df_receipt), calculate the number of days elapsed since Monday of the week in question and display 10 items together with the sales date and the previous Monday (note that sales_ymd holds data in numerical form).


---
> P-075: Randomly extract 1% of the data from the customer data (df_customer) and display the first 10 items.

---
> P-076: Extract 10% of the data stratified randomly from the customer data (df_customer) based on the percentage of gender code (gender_cd) and count the number of cases by gender code.

---
> P-077: Sum the sales amounts in the receipt details data (df_receipt) by customer unit and extract outliers of the summed sales amounts. The outliers should be calculated by logarithmising the total sales amount, calculating the mean and standard deviation, and then deviating from the mean by more than 3σ (either the natural logarithm or the ordinary logarithm is acceptable). Display 10 results.


---
> P-078: Sum the sales amount (amount) of the receipt details data (df_receipt) in customer units and extract outliers of the summed sales amount. However, exclude those whose customer ID starts with "Z", as they represent non-members. Outliers are defined as the difference between the first and third quartiles using IQR, and are defined as being below "first quartile - 1.5 x IQR" or above "third quartile + 1.5 x IQR". Show 10 results.

---
> P-079: For each item of product data (df_product), check the number of missing items.

---
> P-080: Create a new product data, deleting all records where any item in the product data (df_product) is missing. Note that the number of items before and after the deletion should be displayed, and also confirm that the number of items has decreased by the number of items checked in 079.

---
> P-081: Create new product data for the missing values of unit_price and unit_cost, supplemented by the average value of each. Note that for the average values, round off values less than one yen (rounding off or rounding to even numbers is acceptable). After completion, also check that there are no missing values for each item.


---
> P-082: Create new product data for missing values of unit_price and unit_cost, supplemented by the median value of each. Note that for the median value, round off the figures to the nearest yen (rounding off or rounding to even numbers is acceptable). After completion, also check that there are no missing values for each item.

---
> P-083: Create new product data for the missing values of unit price and unit cost, supplemented by the median value calculated for each product category sub-category code (category_small_cd). Note that the median value should be rounded to the nearest yen (rounding off or rounding to even numbers is acceptable). After completion, also check that there are no deficiencies for each item.


---
> P-084: Prepare new data for all customers in the customer data (df_customer) by calculating the percentage of the 2019 sales value to the sales value for the whole period. However, if there are no sales results, treat them as 0. Then, extract the calculated percentages that exceed 0 and display the results for 10 cases. Also check that there are no missing data in the data created.

---
> P-085: Create new customer data by linking geocode data (df_geocode) to all customers in the customer data (df_customer) using postcode (postal_cd). However, if multiple longitude (longitude) and latitude (latitude) information is associated with a single postcode (postal_cd), calculate and use the average values of longitude (longitude) and latitude (latitude). Also, display 10 results to check the creation results.

---
> P-086: For the customer data with latitude(latitude) and longitude(longitude) created in 085, combine it with shop data (df_store) using the member application shop code (application_store_cd) as key. Calculate the distance (in km) between the application shop and the customer address using the latitude(latitude) and longitude information (longitude) of the application shop and the latitude and longitude of the customer address (address), and display it together with the customer ID (customer_id), customer address and shop address (address). Display the information together with the customer ID (customer_id), customer address (address) and store address (address). The following simple formula should be used as the calculation formula, but libraries using other highly accurate methods are also acceptable. Display 10 results.


$$
\mbox{Latitude（radians）}：\phi \\
\mbox{Longitude（radians）}：\lambda \\
\mbox{Distance }L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

---
> P-087: In the customer data (df_customer), the same customer is registered more than once, e.g. due to applications at different shops. Create the named customer data, in which customers with the same name (customer_name) and postcode (postal_cd) are regarded as the same customer and are collated so that there is one record per customer, and calculate the number of customer data, the number of named customer data and the number of duplicates. However, for the same customer, the one with the highest total sales amount shall be retained, and for customers with the same total sales amount or no sales record, the one with the lowest customer ID (customer_id) number shall be retained.


---
> P-088: Based on the data created in 087, create data with integrated name IDs for the customer data. However, the integrated name ID shall be assigned according to the following specifications.
>
> - Non-duplicated customers: set customer ID (customer_id)
> - Duplicate customers: set the customer ID of the record extracted in the previous question
> 
> The difference between the number of unique customer IDs and the number of unique integration name-drop IDs shall also be checked.

---
> P - Quiet: df_customer_1, df_customer_n are not used and should be deleted.

In [ ]:
del df_customer_1
del df_customer_n

---
> P-089: We want to split customers with sales records into training data and test data for building a forecasting model. Split the data randomly in the ratio of 8:2 respectively.

---
> P-090: The receipt details data (df_receipt) has data from 1 Jan 2017 to 31 Oct 2019. Create three sets of data for building time-series models, 12 months for training and 6 months for testing, by aggregating the sales amount (amount) on a monthly basis.

---
> P-091: For each customer in the customer data (df_customer), under-sample the number of customers with sales performance and the number of customers without sales performance so that the ratio is 1:1.

---
> P-092: Normalise to third normal form for gender in customer data (df_customer).

---
> P-093: Product data (df_product) only holds the code values for each category, but not the category names. Combine with category data (df_category) and denormalise to create new product data that holds category names.

---
> P-094: File out the product data with category names created in 093 with the following specifications.
>
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |CSV (comma separated)|With|UTF-8|
> 
> The path of the file output destination shall be as follows
> 
> |Output|
> |:--:|
> |./data|

---
> P-095: Output the product data with category names created in 093 to a file with the following specifications.
>
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |CSV (comma separated)|With|CP932|
> 
> The path of the file output destination shall be as follows
> 
> |Output|
> |:--:|
> |./data|

---
> P-096: Output the product data with category names created in 093 to a file with the following specifications.
>
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |CSV (comma separated)|without|UTF-8|
> 
> The path of the file output destination shall be as follows
> 
> |Output|
> |:--:|
> |./data|

---
> P-097: Load the file in the following format created in 094, display three data items and check that they have been imported correctly.
> 
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |CSV (comma separated)|With|UTF-8|

---
> P-098: Load the file in the following format created in 096, display three data items and check that they are imported correctly.
> 
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |CSV (comma separated)|Without|UTF-8|

---
> P-099: Output the product data with category names created in 093 to a file with the following specifications.
>
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |TSV (Tab Separated)|With|UTF-8|
> 
> The path of the file output destination shall be as follows
> 
> |Output|
> |:--:|
> |./data|

---
> P-100: Load the file in the following format created in 099, display three items of data and check that they have been imported correctly.
> 
> |File format|With/without header|Character encoding|
> |:--:|:--:|:--:|
> |TSV (Tab Separated)|With|UTF-8|

# This is the end of the 100. Well done!